## Chat Models - Output Parsing

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

"Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
"Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.
And then one optional one:

"Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.


In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
model = ChatOpenAI()

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")

class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")

parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
template = "Answer the user query.\n{format_instructions}\n{query}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])


messages = chat_prompt.invoke({
    "query": "What is a really funny joke about Python programming?",
    "format_instructions": parser.get_format_instructions()
})

In [ ]:
result = model.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.setup)
    print(joke_object.punchline)
except Exception as e:
    print(e)

Why do Python programmers prefer dark mode?
Because light attracts bugs!


In [ ]:
structured_llm = model.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")

In [ ]:
print(result.punchline)
print(type(result))

Because light attracts bugs!
<class '__main__.Joke'>
